This script reproduces what the main.py function does but divided into its parts so we can visualize the whole process

In [9]:
import os
import torch
import torch.nn as nn
import numpy as np
import argparse
import pickle
from ucimlrepo import fetch_ucirepo 

In [10]:
from src.dataset import Data_handling
from src.weakener import Weakener
from src.model import MLP
from utils.datasets_generation import generate_dataset
import utils.losses as losses
from utils.train_test_loop import train_and_evaluate

In [11]:
reps = 10
dataset_base_path = 'Datasets/weak_datasets'
dataset = 'mnist'
corruption = 'Noisy_Patrini_MNIST'
corr_p = 0.7
corr_n = None
loss_type = 'Forward'

for i in range(reps):
        generate_dataset(dataset=dataset,corruption=corruption,corr_p=corr_p,repetitions=i)


In [12]:


for i in range(reps):
    base_dir = "Datasets/weak_datasets"
    if corr_n is not None:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p_+{corr_p}p_-{corr_n}')
    else:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p{corr_p}')
    f = open(folder_path + f'/Dataset_{i}.pkl','rb')
    Data,Weak = pickle.load(f)
    f.close()

    
    
    if loss_type == 'Backward':
        loss_fn = losses.FwdBwdLoss(Weak.Y,np.eye(Weak.c))
    elif loss_type == 'Forward':
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'Forward_opt':
        B = Weak.M @ torch.inverse(self.M.T @ torch.inverse(torch.diag(self.pest)) @ self.M ) @ self.M.T @ torch.inverse(torch.diag(self.pest))
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'EM':
        loss_fn = losses.EMLoss(Weak.M)
    elif loss_type == 'LBL':
        loss_fn = losses.LBLoss()
    elif loss_type == 'Backward_opt':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt,np.eye(Weak.c))
    elif loss_type == 'Backward_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_conv,np.eye(Weak.c))
    elif loss_type == 'Backward_opt_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt_conv,np.eye(Weak.c))
    elif loss_type == 'OSL':
        loss_fn = losses.OSLCELoss()
    
    if loss_type == 'OSL':
        Data.include_weak(Weak.w)
    else:
        Data.include_weak(Weak.z)

    
    trainloader,testloader = Data.get_dataloader(weak_labels = 'weak')

    print(Data.num_features)


    #trainloader,testloader = Data.get_dataloader()
    mlp = MLP(Data.num_features, [], Weak.c, dropout_p=0, bn = False, activation='id')
    optim = torch.optim.Adam(mlp.parameters(), lr=1e-3,betas=(0.9, 0.999), eps=1e-8)
    mlp,results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss_fn,corr_p=corr_p,num_epochs=10,sound=1,rep=i)






# Bwd = FwdBwdLoss(pinv(M),I_c)
# Fwd = FwdBwdLoss(I_d,M)

784
Epoch 1/10: Train Loss: 2.6349, Train Acc: 0.7526, Test Acc: 0.7704, Train Detached Loss: 1.4412, Test Detached Loss: 1.4818, Learning Rate: 0.001000
Epoch 2/10: Train Loss: 2.2664, Train Acc: 0.7667, Test Acc: 0.7658, Train Detached Loss: 3.0852, Test Detached Loss: 3.1539, Learning Rate: 0.001000
Epoch 3/10: Train Loss: 2.2729, Train Acc: 0.7676, Test Acc: 0.7772, Train Detached Loss: 3.7580, Test Detached Loss: 3.8099, Learning Rate: 0.001000
Epoch 4/10: Train Loss: 2.0868, Train Acc: 0.7756, Test Acc: 0.7714, Train Detached Loss: 4.2651, Test Detached Loss: 4.3548, Learning Rate: 0.001000
Epoch 5/10: Train Loss: 2.1131, Train Acc: 0.7739, Test Acc: 0.7819, Train Detached Loss: 4.1922, Test Detached Loss: 4.1603, Learning Rate: 0.001000
Epoch 6/10: Train Loss: 2.0643, Train Acc: 0.7791, Test Acc: 0.7902, Train Detached Loss: 4.2147, Test Detached Loss: 4.2658, Learning Rate: 0.001000
Epoch 7/10: Train Loss: 2.0360, Train Acc: 0.7798, Test Acc: 0.7819, Train Detached Loss: 4.6557

KeyboardInterrupt: 

In [13]:
results

,epoch,train_loss,train_acc,test_acc,train_detached_loss,test_detached_loss,optimizer,loss_fn,repetition,initial_lr,actual_lr,corr_p
0,1,2.634868,0.752617,0.7704,1.441179,1.481786,Adam,None,0,0.001,0.001,0.7
1,2,2.266381,0.766717,0.7658,3.085225,3.153918,Adam,None,0,0.001,0.001,0.7
2,3,2.272940,0.767583,0.7772,3.757974,3.809920,Adam,None,0,0.001,0.001,0.7
3,4,2.086789,0.775650,0.7714,4.265095,4.354797,Adam,None,0,0.001,0.001,0.7
4,5,2.113070,0.773867,0.7819,4.192219,4.160255,Adam,None,0,0.001,0.001,0.7
5,6,2.064319,0.779083,0.7902,4.214673,4.265758,Adam,None,0,0.001,0.001,0.7
6,7,2.035951,0.779767,0.7819,4.655750,4.731796,Adam,None,0,0.001,0.001,0.7
7,8,2.018933,0.781983,0.7862,5.235042,5.325461,Adam,None,0,0.001,0.001,0.7
8,9,1.992198,0.793433,0.7998,4.499726,4.529219,Adam,None,0,0.001,0.001,0.7
9,10,2.035942,0.786933,0.7822,5.766142,5.911747,Adam,None,0,0.001,0.001,0.7


In [25]:
np.max(Weak.M - B.detach().numpy())

0.10000000000000002

In [18]:
pest = torch.from_numpy(Weak.generate_wl_priors())
tm = torch.from_numpy(Weak.M)
B = tm @ torch.inverse(tm.T @ torch.inverse(torch.diag(pest)) @ tm) @ tm.T @ torch.inverse(torch.diag(pest))
B.sum(dim=0)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], dtype=torch.float64)

In [22]:
Weak.Y-Weak.Y_conv

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
reps = 10
dataset_base_path = 'Datasets/weak_datasets'
dataset = 'mnist'
corruption = 'Noisy_Patrini_MNIST'
corr_p = 0.7
corr_n = None
loss_type = 'Backward'
for i in range(1):
    base_dir = "Datasets/weak_datasets"
    if corr_n is not None:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p_+{corr_p}p_-{corr_n}')
    else:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p{corr_p}')
    f = open(folder_path + f'/Dataset_{i}.pkl','rb')
    Data,Weak = pickle.load(f)
    f.close()

    
    
    if loss_type == 'Backward':
        loss_fn = losses.FwdBwdLoss(Weak.Y,np.eye(Weak.c))
    elif loss_type == 'Forward':
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'Forward_opt':
        B = Weak.M @ torch.inverse(self.M.T @ torch.inverse(torch.diag(self.pest)) @ self.M ) @ self.M.T @ torch.inverse(torch.diag(self.pest))
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'EM':
        loss_fn = losses.EMLoss(Weak.M)
    elif loss_type == 'LBL':
        loss_fn = losses.LBLoss()
    elif loss_type == 'Backward_opt':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt,np.eye(Weak.c))
    elif loss_type == 'Backward_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_conv,np.eye(Weak.c))
    elif loss_type == 'Backward_opt_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt_conv,np.eye(Weak.c))
    elif loss_type == 'OSL':
        loss_fn = losses.OSLCELoss()
    
    if loss_type == 'OSL':
        Data.include_weak(Weak.w)
    else:
        Data.include_weak(Weak.z)

    
    trainloader,testloader = Data.get_dataloader(weak_labels = 'weak')

    print(Data.num_features)


    #trainloader,testloader = Data.get_dataloader()
    mlp = MLP(Data.num_features, [], Weak.c, dropout_p=0, bn = False, activation='id')
    optim = torch.optim.Adam(mlp.parameters(), lr=1e-3,betas=(0.9, 0.999), eps=1e-8)
    mlp,results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss_fn,corr_p=corr_p,num_epochs=10,sound=1,rep=i)






# Bwd = FwdBwdLoss(pinv(M),I_c)
# Fwd = FwdBwdLoss(I_d,M)

784
Epoch 1/10: Train Loss: 2.4305, Train Acc: 0.8500, Test Acc: 0.8799, Train Detached Loss: 0.4478, Test Detached Loss: 0.4365, Learning Rate: 0.001000
Epoch 2/10: Train Loss: 2.1110, Train Acc: 0.8773, Test Acc: 0.8975, Train Detached Loss: 0.5769, Test Detached Loss: 0.5284, Learning Rate: 0.001000
Epoch 3/10: Train Loss: 2.0091, Train Acc: 0.8852, Test Acc: 0.8954, Train Detached Loss: 0.7091, Test Detached Loss: 0.6975, Learning Rate: 0.001000
Epoch 4/10: Train Loss: 1.9082, Train Acc: 0.8916, Test Acc: 0.9044, Train Detached Loss: 0.7677, Test Detached Loss: 0.7466, Learning Rate: 0.001000
Epoch 5/10: Train Loss: 1.8812, Train Acc: 0.8938, Test Acc: 0.8974, Train Detached Loss: 1.0000, Test Detached Loss: 0.9904, Learning Rate: 0.001000
Epoch 6/10: Train Loss: 1.8379, Train Acc: 0.8967, Test Acc: 0.9083, Train Detached Loss: 0.9244, Test Detached Loss: 0.9052, Learning Rate: 0.001000
Epoch 7/10: Train Loss: 1.8364, Train Acc: 0.8965, Test Acc: 0.9135, Train Detached Loss: 0.9985

In [27]:
reps = 10
dataset_base_path = 'Datasets/weak_datasets'
dataset = 'mnist'
corruption = 'Noisy_Patrini_MNIST'
corr_p = 0.7
corr_n = None
loss_type = 'Backward_conv'
for i in range(1):
    base_dir = "Datasets/weak_datasets"
    if corr_n is not None:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p_+{corr_p}p_-{corr_n}')
    else:
        folder_path = os.path.join(base_dir, f'{dataset}_{corruption}_p{corr_p}')
    f = open(folder_path + f'/Dataset_{i}.pkl','rb')
    Data,Weak = pickle.load(f)
    f.close()

    
    
    if loss_type == 'Backward':
        loss_fn = losses.FwdBwdLoss(Weak.Y,np.eye(Weak.c))
    elif loss_type == 'Forward':
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'Forward_opt':
        B = Weak.M @ torch.inverse(self.M.T @ torch.inverse(torch.diag(self.pest)) @ self.M ) @ self.M.T @ torch.inverse(torch.diag(self.pest))
        loss_fn = losses.FwdBwdLoss(np.eye(Weak.d),Weak.M)
    elif loss_type == 'EM':
        loss_fn = losses.EMLoss(Weak.M)
    elif loss_type == 'LBL':
        loss_fn = losses.LBLoss()
    elif loss_type == 'Backward_opt':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt,np.eye(Weak.c))
    elif loss_type == 'Backward_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_conv,np.eye(Weak.c))
    elif loss_type == 'Backward_opt_conv':
        loss_fn = losses.FwdBwdLoss(Weak.Y_opt_conv,np.eye(Weak.c))
    elif loss_type == 'OSL':
        loss_fn = losses.OSLCELoss()
    
    if loss_type == 'OSL':
        Data.include_weak(Weak.w)
    else:
        Data.include_weak(Weak.z)

    
    trainloader,testloader = Data.get_dataloader(weak_labels = 'weak')

    print(Data.num_features)


    #trainloader,testloader = Data.get_dataloader()
    mlp = MLP(Data.num_features, [], Weak.c, dropout_p=0, bn = False, activation='id')
    optim = torch.optim.Adam(mlp.parameters(), lr=1e-3,betas=(0.9, 0.999), eps=1e-8)
    mlp,results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss_fn,corr_p=corr_p,num_epochs=10,sound=1,rep=i)



784
Epoch 1/10: Train Loss: 2.4305, Train Acc: 0.8500, Test Acc: 0.8799, Train Detached Loss: 0.4478, Test Detached Loss: 0.4365, Learning Rate: 0.001000
Epoch 2/10: Train Loss: 2.1110, Train Acc: 0.8773, Test Acc: 0.8975, Train Detached Loss: 0.5769, Test Detached Loss: 0.5284, Learning Rate: 0.001000
Epoch 3/10: Train Loss: 2.0091, Train Acc: 0.8852, Test Acc: 0.8954, Train Detached Loss: 0.7091, Test Detached Loss: 0.6975, Learning Rate: 0.001000
Epoch 4/10: Train Loss: 1.9082, Train Acc: 0.8916, Test Acc: 0.9044, Train Detached Loss: 0.7677, Test Detached Loss: 0.7466, Learning Rate: 0.001000
Epoch 5/10: Train Loss: 1.8812, Train Acc: 0.8938, Test Acc: 0.8974, Train Detached Loss: 1.0000, Test Detached Loss: 0.9904, Learning Rate: 0.001000
Epoch 6/10: Train Loss: 1.8379, Train Acc: 0.8967, Test Acc: 0.9083, Train Detached Loss: 0.9244, Test Detached Loss: 0.9052, Learning Rate: 0.001000
Epoch 7/10: Train Loss: 1.8364, Train Acc: 0.8965, Test Acc: 0.9135, Train Detached Loss: 0.9985

In [ ]:

res_dir = f"Results/{dataset}_{corruption}"
os.makedirs(folder_path, exist_ok=True)
if corr_n is not None:
    file_name = f'{loss_type}_p_+{corr_p}p_-{corr_n}.csv'
else:
    file_name = f'{loss_type}_p_+{corr_p}p_-{corr_n}.csv'
file_path = os.path.join(res_dir, file_name)
results.to_csv(file_path, index=False)

print(f'DataFrame saved as CSV at: {file_path}')


DataFrame saved as CSV at: Results\image_pll_p0.5\Forward.csv


In [ ]:
results

,epoch,train_loss,train_acc,test_acc,train_detached_loss,test_detached_loss,optimizer,loss_fn,repetition,initial_lr,actual_lr,corr_p
0,1,3.469290,0.196429,0.142857,0.066955,0.090892,Adam,FwdBwdLoss,9,0.001,0.001,0.5
1,2,3.441603,0.285714,0.214286,0.065961,0.089786,Adam,FwdBwdLoss,9,0.001,0.001,0.5
2,3,3.413559,0.285714,0.261905,0.065322,0.088971,Adam,FwdBwdLoss,9,0.001,0.001,0.5
3,4,3.388708,0.285714,0.285714,0.064642,0.088314,Adam,FwdBwdLoss,9,0.001,0.001,0.5
4,5,3.371164,0.285714,0.285714,0.063951,0.087665,Adam,FwdBwdLoss,9,0.001,0.001,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,2.707886,0.583333,0.500000,0.035038,0.062313,Adam,FwdBwdLoss,9,0.001,0.001,0.5
96,97,2.713032,0.607143,0.500000,0.034726,0.061899,Adam,FwdBwdLoss,9,0.001,0.001,0.5
97,98,2.714207,0.595238,0.547619,0.034487,0.061459,Adam,FwdBwdLoss,9,0.001,0.001,0.5
98,99,2.714451,0.601190,0.547619,0.034288,0.061044,Adam,FwdBwdLoss,9,0.001,0.001,0.5
